<a href="https://colab.research.google.com/github/guilhermeguy/final-report-puc/blob/main/xray_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import maplotlib.pyplot as plt
%matplotlib inline